# Part 1: Document similarity

In [6]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    tokens = nltk.word_tokenize(doc)
    pos = nltk.pos_tag(tokens)

    tags = [tag[1] for tag in pos]
    wntag = [convert_tag(tag) for tag in tags]

    ans = list(zip(tokens, wntag))
    sets = [wn.synsets(a, b) for a, b in ans]
    
    return [val[0] for val in sets if len(val) > 0]


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    s = []
    for i in s1:
        r = []
        scores = [x for x in [i.path_similarity(j) for j in s2] if x is not None]
        if scores:
            s.append(max(scores))
    
    return sum(s) / len(s)


def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

document_path_similarity("It's great to be back", "It's good to be back")

0.8333333333333333

## test document path similarity

In [7]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

In [9]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('data/paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


### most similar docs

In [12]:
def most_similar_docs():
    
    temp = paraphrases.copy()
    temp["similarity"] = temp.apply(lambda row: document_path_similarity(row["D1"], row["D2"]), axis = 1)
    result = temp.loc[temp["similarity"] == temp["similarity"].max()].squeeze().values    
    return result[1], result[2], result[3]

most_similar_docs()

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.9590643274853801)

## label accuracy

In [14]:
def label_accuracy():
    from sklearn.metrics import accuracy_score
    def get_label(row):
        if row['similarity'] > 0.75:
            row['label'] = 1
        else:
            row['label'] = 0
        return row

    temp = paraphrases.copy()
    temp['similarity'] = temp.apply(lambda row: document_path_similarity(row['D1'], row['D2']), axis=1)
    temp = temp.apply(get_label, axis=1)
    
    return accuracy_score(temp['Quality'], temp['label'])

label_accuracy()

0.7

## Part 2: Topic Modelling

In [15]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('data/newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [16]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

# Your code here:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, id2word = id_map, num_topics = 10, passes = 25, random_state = 42)

### lda topics

In [17]:
def lda_topics():
    return ldamodel.print_topics(num_topics = 10, num_words = 10)

lda_topics()

[(0,
  '0.070*"edu" + 0.048*"com" + 0.020*"don" + 0.013*"want" + 0.012*"people" + 0.012*"know" + 0.011*"food" + 0.009*"article" + 0.009*"just" + 0.009*"blood"'),
 (1,
  '0.027*"game" + 0.025*"year" + 0.020*"team" + 0.016*"games" + 0.015*"play" + 0.012*"good" + 0.012*"season" + 0.011*"win" + 0.010*"don" + 0.010*"players"'),
 (2,
  '0.027*"drive" + 0.026*"scsi" + 0.022*"card" + 0.016*"does" + 0.016*"use" + 0.014*"cable" + 0.013*"bus" + 0.013*"controller" + 0.013*"mac" + 0.013*"board"'),
 (3,
  '0.032*"ground" + 0.026*"current" + 0.025*"time" + 0.023*"use" + 0.018*"used" + 0.016*"box" + 0.013*"circuit" + 0.012*"way" + 0.011*"power" + 0.011*"left"'),
 (4,
  '0.016*"don" + 0.015*"just" + 0.015*"day" + 0.015*"hockey" + 0.014*"know" + 0.014*"time" + 0.013*"does" + 0.013*"bike" + 0.013*"little" + 0.011*"got"'),
 (5,
  '0.027*"thanks" + 0.022*"know" + 0.021*"does" + 0.017*"new" + 0.016*"mail" + 0.014*"help" + 0.012*"info" + 0.011*"used" + 0.011*"available" + 0.011*"apple"'),
 (6,
  '0.021*"god"

### topic distribution

In [18]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [19]:
def topic_distribution():
    
    transformed_doc = vect.transform(new_doc)
    corpus = gensim.matutils.Sparse2Corpus(transformed_doc, documents_columns = False)
    doc_topics = ldamodel.get_document_topics(corpus)

    topic_dist = []
    for val in list(doc_topics):
        for val2 in val:
            topic_dist.append(val2)
    
    return topic_dist

topic_distribution()

[(0, 0.020007791),
 (1, 0.020009624),
 (2, 0.020002503),
 (3, 0.02000374),
 (4, 0.020016167),
 (5, 0.020002745),
 (6, 0.020010378),
 (7, 0.81993306),
 (8, 0.020004725),
 (9, 0.020009309)]

### Topic names

In [23]:
def topic_names():
    
    topic_names = ['Health', 'Automobiles', 'Government', 'Travel', 'Computers & IT', 'Sports', 'Business', 'Society & Lifestyle', 'Region', 'Education']
    topics = lda_topics()
    results = []
    for _, dis in topics:
        #print(dis)
        similarity = []
        for topic in topic_names:
            similarity.append(document_path_similarity(dis, topic))
        best_topic = sorted(zip(similarity, topic_names))[-1][1]
        results.append(best_topic)
    return results

topic_names()

['Society & Lifestyle',
 'Education',
 'Education',
 'Society & Lifestyle',
 'Automobiles',
 'Education',
 'Society & Lifestyle',
 'Region',
 'Society & Lifestyle',
 'Automobiles']